In [38]:
import pandas as pd
import duckdb
from pathlib import Path
# --- Importe suas classes de sistema ---
from power.systems import B3EOL, IEEE118EOL, B6L8EOL
import numpy as np

In [6]:
results_path = Path("results_parquet/2025-10-19_22-54-02")

# Carregar dados Parquet

In [12]:
df_angulos = pd.read_parquet(f"{results_path}/angulos.parquet")
df_cargas_individuais = pd.read_parquet(f"{results_path}/cargas_individuais.parquet")
df_corte_carga = pd.read_parquet(f"{results_path}/corte_carga.parquet")
df_corte_carga_detalhado = pd.read_parquet(f"{results_path}/corte_carga_detalhado.parquet")
df_curtailment_detalhado = pd.read_parquet(f"{results_path}/curtailment_detalhado.parquet")
df_fluxo = pd.read_parquet(f"{results_path}/fluxo.parquet")
df_geracao = pd.read_parquet(f"{results_path}/geracao.parquet")
df_limites_angulo = pd.read_parquet(f"{results_path}/limites_angulo.parquet")
df_limites_corte = pd.read_parquet(f"{results_path}/limites_corte.parquet")
df_limites_fluxo = pd.read_parquet(f"{results_path}/limites_fluxo.parquet")
df_limites_geracao = pd.read_parquet(f"{results_path}/limites_geracao.parquet")
df_lmp = pd.read_parquet(f"{results_path}/lmp.parquet")
df_perdas_barra = pd.read_parquet(f"{results_path}/perdas_barra.parquet")
df_perdas_linha = pd.read_parquet(f"{results_path}/perdas_linha.parquet")
df_sumario_geral = pd.read_parquet(f"{results_path}/sumario_geral.parquet")

# --- Primeira Inspeção ---
print("Dados Carregados com Sucesso!")


Dados Carregados com Sucesso!


# Corte de carga médio

In [53]:
# Pra cada CTG, quais barras tiveram maior corte de carga médio?
# Ou seja, dos 10 diferentes cenários, vamos calcular a média do corte de carga por ctg

mapa_carga_barra = {}
for net_class in [B3EOL, IEEE118EOL, B6L8EOL]:
    net_temp = net_class()
    mapa_carga_barra[net_temp.name] = {load.id: load.bus.id for load in net_temp.loads}

df_corte_carga_detalhado['barra_id'] = df_corte_carga_detalhado.apply(
    lambda row: mapa_carga_barra[row['sistema']].get(row['carga_id']),
    axis=1
).astype('Int64') # 'Int64' (com 'I' maiúsculo) lida melhor com possíveis valores nulos

df_corte_carga_detalhado.dropna(subset=['barra_id'], inplace=True)

# Cálculo da média por barra
df_ctg_only = df_corte_carga_detalhado[df_corte_carga_detalhado['contingencia'] != 'BASE_CASE'].copy()
df_ctg_only['contingencia'] = pd.to_numeric(df_ctg_only['contingencia'])

corte_medio_por_barra = df_ctg_only.groupby(
    ['sistema', 'contingencia', 'barra_id']
)['carga_cortada_mw'].mean().reset_index()

corte_medio_por_barra = corte_medio_por_barra[corte_medio_por_barra['carga_cortada_mw'] > 0.001]

# =============================================================================
# FASE 3: APRESENTAÇÃO DOS RANKINGS (POR SISTEMA, SEM FILTRO "TOP N")
# =============================================================================
print("\n" + "="*70)
print(" RANKING DE BARRAS MAIS AFETADAS POR CORTE DE CARGA MÉDIO (MW) ")
print("="*70)

# Itera sobre cada sistema encontrado nos resultados
for sistema in sorted(corte_medio_por_barra['sistema'].unique()):
    print(f"\n\n--- SISTEMA: {sistema} ---")
    
    # Filtra os dados apenas para o sistema atual
    df_sistema = corte_medio_por_barra[corte_medio_por_barra['sistema'] == sistema]

    if df_sistema.empty:
        print("Nenhum corte de carga significativo foi registrado para este sistema.")
        continue

    # A MUDANÇA ESTÁ AQUI: Apenas ordena os resultados, sem usar .head()
    # 1. Ordena por contingência (para agrupar visualmente)
    # 2. Ordena por corte de carga (para ranquear dentro de cada grupo de contingência)
    ranking_completo_sistema = df_sistema.sort_values(
        # by=['contingencia', 'carga_cortada_mw'],
        by=['carga_cortada_mw'],
        # ascending=[True, False]
        ascending=[False]
    )
    
    print("Ranking completo de Barras Afetadas por Contingência:")
    # .to_string() garante que a tabela seja impressa por completo e bem alinhada
    print(ranking_completo_sistema[['contingencia', 'barra_id', 'carga_cortada_mw']].to_string(index=False))

print("\n\n" + "="*70)
print(" ANÁLISE CONCLUÍDA ")
print("="*70)


 RANKING DE BARRAS MAIS AFETADAS POR CORTE DE CARGA MÉDIO (MW) 


--- SISTEMA: B3_EOLIC ---
Ranking completo de Barras Afetadas por Contingência:
 contingencia  barra_id  carga_cortada_mw
            3         3          3.408393
            2         3          0.663213


--- SISTEMA: B6L8_EOLIC ---
Ranking completo de Barras Afetadas por Contingência:
 contingencia  barra_id  carga_cortada_mw
            6         6         20.246675
            7         6         13.916796
            5         6         10.335337
            4         6          6.213565
            4         5          4.514676
            7         5          2.567020
            8         6          2.100046
            2         6          0.729868


--- SISTEMA: IEEE_118_Eolic ---
Ranking completo de Barras Afetadas por Contingência:
 contingencia  barra_id  carga_cortada_mw
           84        59        211.803190
           36        59        183.214154
           77        59        137.157487
         

In [18]:
df_corte_carga_detalhado

,sistema,cenario,contingencia,carga_id,demanda_nominal_mw,carga_cortada_mw,carga_atendida_mw,barra_id
0,B6L8_EOLIC,0,BASE_CASE,5,42.946570,1.610498,41.336072,6
1,B6L8_EOLIC,0,2,5,42.946570,1.354444,41.592126,6
2,B6L8_EOLIC,0,4,4,32.437247,7.070539,25.366708,5
3,B6L8_EOLIC,0,4,5,42.946570,8.490379,34.456191,6
4,B6L8_EOLIC,0,5,5,42.946570,13.074744,29.871826,6
...,...,...,...,...,...,...,...,...
24482,B3_EOLIC,6,3,1,9.503339,3.046059,6.457281,3
24483,B3_EOLIC,7,2,1,10.392581,0.417333,9.975248,3
24484,B3_EOLIC,7,3,1,10.392581,5.428227,4.964354,3
24485,B3_EOLIC,9,2,1,10.561731,0.586484,9.975248,3


# Curtailment Médio

In [ ]:
# Filtra para remover os casos base, pois queremos analisar apenas o impacto das contingências
df_ctg_curtailment = df_curtailment_detalhado[
    df_curtailment_detalhado['contingencia'] != 'BASE_CASE'
].copy()

# A coluna 'curtailment_mw' já está disponível e é o valor que queremos somar/tirar a média.
curtailment_medio = df_ctg_curtailment.groupby(
    ['sistema', 'cenario', 'contingencia']
)['curtailment_mw'].sum().reset_index()

# Agora, calculamos a MÉDIA DESSES TOTAIS sobre os 10 cenários
curtailment_medio_final = curtailment_medio.groupby(
    ['sistema', 'contingencia']
)['curtailment_mw'].mean().reset_index()

# Filtra para mostrar apenas os casos onde houve de fato um curtailment médio significativo
df_curtailment_final = curtailment_medio_final[
    curtailment_medio_final['curtailment_mw'] > 0.001
].copy()


# --- 2. APRESENTAÇÃO DOS RANKINGS (POR SISTEMA) ---
print("\n" + "="*70)
print(" RANKING DE CURTAILMENT MÉDIO TOTAL POR CONTINGÊNCIA (MW) ")
print(" (Média de curtailment TOTAL do sistema sobre os 10 cenários) ")
print("="*70)

# Itera sobre cada sistema para apresentar o relatório separadamente
for sistema, df_sistema in df_curtailment_final.groupby('sistema'):
    print(f"\n\n--- SISTEMA: {sistema} ---")
    
    # Ordena as contingências pelo valor médio de curtailment (do maior para o menor)
    ranking_curtailment = df_sistema.sort_values(
        by='curtailment_mw',
        ascending=False
    )
    
    print("Média de Curtailment Total (MW):")
    # Usa to_string() para garantir a exibição completa e alinhada
    print(ranking_curtailment[['contingencia', 'curtailment_mw']].to_string(index=False))

print("\n\n" + "="*70)
print(" ANÁLISE CONCLUÍDA ")
print("="*70)


 RANKING DE CURTAILMENT MÉDIO TOTAL POR CONTINGÊNCIA (MW) 
 (Média de curtailment TOTAL do sistema sobre os 10 cenários) 


--- SISTEMA: B3_EOLIC ---
Média de Curtailment Total (MW):
contingencia  curtailment_mw
           2        5.144425
           1        2.137947
           3        2.137947


--- SISTEMA: B6L8_EOLIC ---
Média de Curtailment Total (MW):
contingencia  curtailment_mw
           7       19.115475
           1        7.440873
           8        3.871347
           5        3.336701
           2        1.243214
           3        0.962653


 ANÁLISE CONCLUÍDA 


# Qual LT é mais impactante?

In [29]:
# Seu DataFrame carregado: df_limites_fluxo

# 1. Garante a robustez do tipo de dado, lidando com possíveis Nulos (None)
#    Isso é essencial, pois 'None' na extração dual pode causar o erro de tipo no Pandas.
df_limites_fluxo['limite_superior_dual'] = pd.to_numeric(df_limites_fluxo['limite_superior_dual'], errors='coerce')
df_limites_fluxo['limite_inferior_dual'] = pd.to_numeric(df_limites_fluxo['limite_inferior_dual'], errors='coerce')

# 2. Calcula o custo de congestionamento
#    Soma o valor absoluto dos duais (superior e inferior), tratando os NaNs como zero.
df_limites_fluxo['custo_congestao'] = (
    df_limites_fluxo['limite_superior_dual'].fillna(0).abs() +
    df_limites_fluxo['limite_inferior_dual'].fillna(0).abs()
)

# 3. Cálculo da Média e Agregação
# Agrupa por sistema e linha para calcular o custo médio de congestionamento (em todos os cenários/CTGs)
impacto_medio_lt = df_limites_fluxo.groupby(['sistema', 'linha_id'])['custo_congestao'].mean().reset_index()

# 4. Apresentação do Ranking
print("\n" + "="*80)
print(" RANKING DE LINHAS DE TRANSMISSÃO (LTs) MAIS IMPACTANTES POR CUSTO MÉDIO DE CONGESTIONAMENTO ($/MWh)")
print(" (Média do valor absoluto dos duais em todos os Cenários/CTGs) ")
print("="*80)

for sistema, df_sistema in impacto_medio_lt.groupby('sistema'):
    print(f"\n--- SISTEMA: {sistema} ---")

    # Ordena as linhas pelo custo de congestionamento (do maior para o menor)
    ranking_lt = df_sistema.sort_values(
        by='custo_congestao',
        ascending=False
    )

    print("Custo Médio de Congestionamento ($/MWh):")
    # Filtra para mostrar apenas as linhas que tiveram algum custo significativo
    ranking_lt_filtrado = ranking_lt[ranking_lt['custo_congestao'] > 0.001]

    print(ranking_lt_filtrado.to_string(index=False))


 RANKING DE LINHAS DE TRANSMISSÃO (LTs) MAIS IMPACTANTES POR CUSTO MÉDIO DE CONGESTIONAMENTO ($/MWh)
 (Média do valor absoluto dos duais em todos os Cenários/CTGs) 

--- SISTEMA: B3_EOLIC ---
Custo Médio de Congestionamento ($/MWh):
 sistema  linha_id  custo_congestao
B3_EOLIC         1       112.888889
B3_EOLIC         3        88.666667

--- SISTEMA: B6L8_EOLIC ---
Custo Médio de Congestionamento ($/MWh):
   sistema  linha_id  custo_congestao
B6L8_EOLIC         6       248.388727
B6L8_EOLIC         7        57.964110
B6L8_EOLIC         1        56.604805
B6L8_EOLIC         5        45.108995
B6L8_EOLIC         4        44.835776
B6L8_EOLIC         3        15.546826

--- SISTEMA: IEEE_118_Eolic ---
Custo Médio de Congestionamento ($/MWh):
       sistema  linha_id  custo_congestao
IEEE_118_Eolic       116       608.049625
IEEE_118_Eolic        78       533.885050
IEEE_118_Eolic        21       470.755438
IEEE_118_Eolic        30       435.784834
IEEE_118_Eolic       163       418.218

# Análise de MVU e MVD

In [49]:
# Garante a ordenação cronológica correta
df_geracao['contingencia'] = df_geracao['contingencia'].astype(str)
df_geracao['cenario'] = pd.to_numeric(df_geracao['cenario'])
df_geracao.sort_values(
    by=['sistema', 'contingencia', 'gerador_id', 'cenario'], 
    inplace=True
)

# 1.1. Cria a coluna Delta P (Pi - P_i-1)
df_geracao['geracao_anterior_mw'] = df_geracao.groupby(
    ['sistema', 'contingencia', 'gerador_id']
)['geracao_mw'].shift(1)
df_geracao['delta_p_mw'] = df_geracao['geracao_mw'] - df_geracao['geracao_anterior_mw']


# =============================================================
# 2. LÓGICA DE SOMA CUMULATIVA CONDICIONAL (O CORE DO PROBLEMA)
# =============================================================

# 2.1. Define a direção da rampa: +1 (Sobe), -1 (Desce), 0 (Estável/NaN)
df_geracao['rampa_direcao'] = np.sign(df_geracao['delta_p_mw'].fillna(0))

# 2.2. Identifica o Ponto de Virada e Cria os Grupos Contínuos
#      O ID do grupo muda toda vez que a direção (+/-) muda.
df_geracao['rampa_grupo'] = (
    (df_geracao['rampa_direcao'] != df_geracao['rampa_direcao'].shift(1))
).cumsum()

# 2.3. Função para calcular o MVU/MVD ACUMULADO: (Max P - Min P em cada grupo)
def calculate_rampa_acumulada(group):
    """Calcula a soma acumulada de delta P dentro de um grupo de direção única."""
    # A soma de todos os pequenos deltas é igual a P_max - P_min
    return group['geracao_mw'].max() - group['geracao_mw'].min()

# 2.4. Calcula a magnitude do pulo ACUMULADO e mapeia de volta
rampa_acumulada = df_geracao.groupby('rampa_grupo').apply(calculate_rampa_acumulada)
df_geracao['pulo_acumulado_total'] = df_geracao['rampa_grupo'].map(rampa_acumulada)

# 2.5. Isola o MVU Acumulado (para subidas) e o MVD Acumulado (para descidas)
df_geracao['mvu_max_acumulado'] = df_geracao.apply(
    lambda row: row['pulo_acumulado_total'] if row['rampa_direcao'] > 0 else 0, axis=1
)
df_geracao['mvd_max_acumulado'] = df_geracao.apply(
    lambda row: row['pulo_acumulado_total'] if row['rampa_direcao'] < 0 else 0, axis=1
)

# =============================================================
# 3. AGREGAÇÃO FINAL (Encontrando o PULO MÁXIMO Acumulado)
# =============================================================

# Agregação: Armazenamos o VALOR MÁXIMO da soma acumulada que ocorreu para cada gerador/CTG.
analise_rampa = df_geracao.groupby(['sistema', 'contingencia', 'gerador_id']).agg(
    # Armazena a SOMA ACUMULADA MÁXIMA de subida
    mvu_max_acumulado=('mvu_max_acumulado', 'max'),
    # Armazena a SOMA ACUMULADA MÁXIMA de descida
    mvd_max_acumulado=('mvd_max_acumulado', 'max'),
).reset_index()


# Filtra para remover casos base e uso insignificante
analise_rampa_final = analise_rampa[
    (analise_rampa['contingencia'] != 'BASE_CASE') &
    (analise_rampa['mvu_max_acumulado'] > 1e-6)
].sort_values(by=['sistema', 'mvu_max_acumulado'], ascending=[True, False])


# --- Relatório Final ---
print("\n" + "="*80)
print(" AVALIAÇÃO DO MÁXIMO PULSO DE RAMPA ACUMULADA (MVu e MVd) POR CONTINGÊNCIA ")
print(" (Representa a soma da rampa até que a direção mude) ")
print("="*80)

for sistema, df_sistema in analise_rampa_final.groupby('sistema'):
    print(f"\n--- SISTEMA: {sistema} ---")
    
    # Ordena o ranking pela exigência máxima acumulada de subida (MVu Acumulado)
    ranking_sistema = df_sistema.sort_values(by='mvu_max_acumulado', ascending=False)
    
    print("Ranking dos Geradores com MAIOR EXIGÊNCIA MÁXIMA ACUMULADA DE RAMPA (MW):")
    print(ranking_sistema[[
        'contingencia', 
        'gerador_id', 
        'mvu_max_acumulado',
        'mvd_max_acumulado',
    ]].to_string(index=False))

C:\Users\gabriel.halfeld\AppData\Local\Temp\ipykernel_10520\637213047.py:36: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  rampa_acumulada = df_geracao.groupby('rampa_grupo').apply(calculate_rampa_acumulada)



 AVALIAÇÃO DO MÁXIMO PULSO DE RAMPA ACUMULADA (MVu e MVd) POR CONTINGÊNCIA 
 (Representa a soma da rampa até que a direção mude) 

--- SISTEMA: B3_EOLIC ---
Ranking dos Geradores com MAIOR EXIGÊNCIA MÁXIMA ACUMULADA DE RAMPA (MW):
contingencia  gerador_id  mvu_max_acumulado  mvd_max_acumulado
           1           2           2.405625           0.123335
           1           1           0.031984           1.518258
           3           1           0.031984           1.518258

--- SISTEMA: B6L8_EOLIC ---
Ranking dos Geradores com MAIOR EXIGÊNCIA MÁXIMA ACUMULADA DE RAMPA (MW):
contingencia  gerador_id  mvu_max_acumulado  mvd_max_acumulado
           2           2          37.037292          15.064809
           3           2          33.443471          13.817982
           8           2          31.286943          11.329699
           6           2          30.288849          11.709318
           3           3          29.222130          40.807344
           2           3          2

In [50]:
for sistema, df_sistema in analise_rampa_final.groupby('sistema'):
    print(f"\n--- SISTEMA: {sistema} ---")
    
    # Ordena o ranking pela exigência máxima acumulada de subida (MVu Acumulado)
    ranking_sistema = df_sistema.sort_values(by='mvd_max_acumulado', ascending=False)
    
    print("Ranking dos Geradores com MAIOR EXIGÊNCIA MÁXIMA ACUMULADA DE RAMPA (MW):")
    print(ranking_sistema[[
        'contingencia', 
        'gerador_id', 
        'mvu_max_acumulado',
        'mvd_max_acumulado',
    ]].to_string(index=False))


--- SISTEMA: B3_EOLIC ---
Ranking dos Geradores com MAIOR EXIGÊNCIA MÁXIMA ACUMULADA DE RAMPA (MW):
contingencia  gerador_id  mvu_max_acumulado  mvd_max_acumulado
           1           1           0.031984           1.518258
           3           1           0.031984           1.518258
           1           2           2.405625           0.123335

--- SISTEMA: B6L8_EOLIC ---
Ranking dos Geradores com MAIOR EXIGÊNCIA MÁXIMA ACUMULADA DE RAMPA (MW):
contingencia  gerador_id  mvu_max_acumulado  mvd_max_acumulado
           3           3          29.222130          40.807344
           2           3          29.210830          40.033743
           1           6          10.676840          24.072917
           2           6          10.676840          24.072917
           3           6          10.676840          24.072917
           4           6          10.676840          24.072917
           6           6          10.676840          24.072917
           5           6          10.676